# Main Wildfire Visualizations

This notebook performs the end-to-end workflow for the wildfire simulation project:

- Download Hansen Global Forest Change rasters for a single 10°×10° tile.
- Crop + downsample the rasters into a tractable simulation grid.
- Construct a simple annual fuel-density time series (2000–2019) from `treecover2000`, `lossyear`, `gain`, and `datamask`.
- Run Monte Carlo wildfire simulations per year under two scenarios:
  - **Baseline** (no mitigation)
  - **Thinning** (uniform fuel reduction)
- Visualize results (risk maps and time-series summaries).
- Quantify thinning effectiveness with confidence intervals and a permutation-test p-value with a Holm–Bonferroni correction across years.

Notes:
- The selected tile is **60N_120W** (wilderness of northern Canada).
- Many cells below can take time to run (especially Monte Carlo loops); keep `n_runs` modest while iterating.

# Imports

These imports cover:
- **Data acquisition**: `requests`, `PIL.Image`, `Path`
- **Computation**: `numpy`
- **Plotting**: `matplotlib`
- **Progress bars**: `tqdm` for long-running loops

Keeping imports near the top makes the notebook easier to rerun from a fresh kernel.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import requests
import io
from pathlib import Path
from tqdm.auto import tqdm

In [ ]:
# Specify the URL of the data tile

# This project uses the Hansen GFC 2019 v1.7 dataset, which is distributed as 10°×10° GeoTIFF tiles.
# The tile identifier is embedded in the filename (e.g., `60N_120W`).

# You can swap this URL to analyze a different tile.
url = (
    "https://storage.googleapis.com/earthenginepartners-hansen/GFC-2019-v1.7/"
    "Hansen_GFC-2019-v1.7_treecover2000_60N_120W.tif"
)

## Hansen Forest Tile Download

We have to specify the URL of the data tile we want to download.
The important part of the URL is the `00N_010E` part at the end.
This specifies the latitude and longitude (in 10-degree units)
of the data tile to download.

The selected data tile is at `60N_120W`, or the wilderness of northern Canada.

In [ ]:
# Load the data. These tiles are large, so we cache to disk and only convert the simulation window
# to a NumPy array (instead of materializing the full 40,000 x 40,000 raster in memory).
Image.MAX_IMAGE_PIXELS = None

def download_if_needed(url: str, out_path: str) -> str:
    p = Path(out_path)
    p.parent.mkdir(parents=True, exist_ok=True)
    if p.exists() and p.stat().st_size > 0:
        return str(p)

    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(p, "wb") as f:
            for chunk in r.iter_content(chunk_size=1024 * 1024):
                if chunk:
                    f.write(chunk)
    return str(p)


def load_tif_window(path: str, row0: int, col0: int, window_px: int) -> np.ndarray:
    im = Image.open(path)
    box = (int(col0), int(row0), int(col0 + window_px), int(row0 + window_px))
    return np.array(im.crop(box))


treecover_path = download_if_needed(url, "data/treecover2000_tile.tif")

# Use PIL to get raster dimensions without loading the full array.
with Image.open(treecover_path) as _im:
    tile_shape = (_im.height, _im.width)

tile_shape

In [ ]:
# Sanity check: tile dimensions (should be 40000x40000)
tile_shape

In [ ]:
# Plot a small part of the whole data tile (from disk-cached file).
preview = load_tif_window(treecover_path, row0=2000, col0=1000, window_px=2000)

plt.figure(figsize=(8, 8))
plt.imshow(preview, cmap='gray')
plt.colorbar(shrink=0.8)
plt.title("Raw Hansen treecover2000 (subset), values 0..100")
plt.show()

In [ ]:
# Import the simulation package
#
# Specification:
# - The `simulation/` package lives in this repository, so we add the repository root to `sys.path`
#   to make `import simulation` work when running the notebook.

# Note: If you install this project as a package instead, you can remove the sys.path manipulation.

import sys
from pathlib import Path

# Make the project root importable so import simulation works
sys.path.append(str(Path().resolve()))

from simulation import FireModelParams, Forest, MonteCarlo, DataCollector

In [ ]:
def mean_pool_2d(a: np.ndarray, out_shape: tuple[int, int]) -> np.ndarray:
    a = np.asarray(a)
    in_rows, in_cols = a.shape
    out_rows, out_cols = out_shape

    if in_rows % out_rows != 0 or in_cols % out_cols != 0:
        raise ValueError("For this simple pooling method, crop so window dims are divisible by out dims")

    r = in_rows // out_rows
    c = in_cols // out_cols
    return a.reshape(out_rows, r, out_cols, c).mean(axis=(1, 3))


# Select a window from the tile and downsample it to a simulation grid.
row0, col0 = 2000, 1000
window_px = 2000
sim_shape = (200, 200)

# Derive matching layer URLs from the treecover URL.
tile_id = url.split("_treecover2000_")[-1].replace(".tif", "")
base = "https://storage.googleapis.com/earthenginepartners-hansen/GFC-2019-v1.7/"

datamask_url = base + f"Hansen_GFC-2019-v1.7_datamask_{tile_id}.tif"
lossyear_url = base + f"Hansen_GFC-2019-v1.7_lossyear_{tile_id}.tif"
gain_url = base + f"Hansen_GFC-2019-v1.7_gain_{tile_id}.tif"

datamask_path = download_if_needed(datamask_url, "data/datamask_tile.tif")
lossyear_path = download_if_needed(lossyear_url, "data/lossyear_tile.tif")
gain_path = download_if_needed(gain_url, "data/gain_tile.tif")

# Load raw windows
window_tree = load_tif_window(treecover_path, row0=row0, col0=col0, window_px=window_px)
window_mask = load_tif_window(datamask_path, row0=row0, col0=col0, window_px=window_px)
window_loss = load_tif_window(lossyear_path, row0=row0, col0=col0, window_px=window_px)
window_gain = load_tif_window(gain_path, row0=row0, col0=col0, window_px=window_px)

# Crop to be divisible for pooling
h = sim_shape[0] * (window_tree.shape[0] // sim_shape[0])
w = sim_shape[1] * (window_tree.shape[1] // sim_shape[1])
window_tree = window_tree[:h, :w]
window_mask = window_mask[:h, :w]
window_loss = window_loss[:h, :w]
window_gain = window_gain[:h, :w]

# Downsample
mask_small = mean_pool_2d(window_mask.astype(float), sim_shape)
# datamask values are categorical; after pooling, interpret >1.5 as "mostly water" and <0.5 as "mostly no data".
water_small = mask_small > 1.5
nodata_small = mask_small < 0.5
land_small = (~water_small) & (~nodata_small)

base_density_2000 = np.clip(mean_pool_2d(window_tree.astype(float), sim_shape) / 100.0, 0.0, 1.0)
base_density_2000 = base_density_2000 * land_small.astype(float)

lossyear_small = mean_pool_2d(window_loss.astype(float), sim_shape)
lossyear_small = np.rint(lossyear_small).astype(int)

# gain is 0/1, but pooling yields fractions. Threshold at 0.5
# (meaning >50% of the pooled pixels gained)
gain_small = mean_pool_2d(window_gain.astype(float), sim_shape) > 0.5

plt.figure(figsize=(6, 6))
plt.imshow(base_density_2000, cmap="Greens")
plt.colorbar(shrink=0.8)
plt.title(f"Density (year 2000) with datamask applied {base_density_2000.shape}")
plt.show()

plt.figure(figsize=(6, 6))
plt.imshow(water_small.astype(int), cmap="Blues")
plt.title("Water mask (1=water)")
plt.show()

# Risk Map time series 
From 2000 to 2019 using additional lossyear and gain masks 

**Assumptions:**
- If a cell has loss in year Y (lossyear == Y-2000), then from that year onward the density is reduced.
- gain is treated as a modest regrowth applied after 2012 in cells marked as gain.
- water/no-data cells (datamask) always have density 0.

In [ ]:
# Build a simple density time series (2000..2019) using lossyear and gain.

p_burned, p_affected = [], []
delta_burned, delta_affected = [], []

def density_for_year(year: int) -> np.ndarray:
    if year < 2000 or year > 2019:
        raise ValueError("year must be in [2000, 2019]")

    d = base_density_2000.copy()

    # Apply loss: lossyear values are 1..19 representing 2001..2019.
    # For a given simulation year, treat any loss event up to that year as removing forest density.
    cutoff = year - 2000
    lost = (lossyear_small > 0) & (lossyear_small <= cutoff)

    # Simple loss model: set to zero (stand-replacing disturbance)
    d[lost] = 0.0

    # Apply gain: only after 2012, modestly increase density in gain cells.
    if year >= 2012:
        d[gain_small] = np.clip(d[gain_small] + 0.25, 0.0, 1.0)

    # Keep land mask enforced
    d *= land_small.astype(float)
    return d


# Visualize density for a few years
years_to_plot = (2000, 2003, 2005, 2008, 2010, 2012, 2015, 2017, 2019)
for y in years_to_plot:
    dy = density_for_year(y)
    plt.figure(figsize=(5, 5))
    plt.imshow(dy, cmap="Greens")
    plt.colorbar(shrink=0.8)
    plt.title(f"Density map (year={y})")
    plt.show()


# Monte Carlo over selected years + a strategy comparison (baseline vs thinning)
params = FireModelParams(
    wind=(1.0, 0.0),
    wind_strength=0.4,
    density_exponent=1.2,
    base_spread=0.7,
)

n_runs = 200

# Include more years to show intermediate changes.
years_to_analyze = (2000, 2003, 2005, 2008, 2010, 2012, 2015, 2017, 2019)

results = {}
for y in tqdm(years_to_analyze, desc="Years"):
    d = density_for_year(y)
    forest = Forest(d)
    mc = MonteCarlo(forest=forest, params=params, n_runs=n_runs, rng=np.random.default_rng(y))

    report = mc.run()
    burned_arr, affected_arr = report.convert_to_arrays()
    risk = mc.risk_map(n_runs=n_runs)
    results[(y, "baseline")] = (burned_arr, affected_arr, risk)

    plt.figure(figsize=(6, 5))
    plt.hist(burned_arr, bins=25)
    plt.title(f"Year {y} baseline: burned fraction distribution")
    plt.xlabel("burned fraction")
    plt.ylabel("count")
    plt.show()

    plt.figure(figsize=(6, 6))
    # Use a reversed colormap so higher risk appears darker
    plt.imshow(risk, cmap="inferno_r", vmin=0.0, vmax=1.0)
    plt.colorbar(shrink=0.8)
    plt.title(f"Year {y} baseline: risk map")
    plt.show()

    burned_mean, burned_lo, burned_hi = DataCollector.calculate_ci95_mean(burned_arr)
    affected_mean, affected_lo, affected_hi = DataCollector.calculate_ci95_mean(affected_arr)
    print(f"Year {y} baseline burned mean+CI:", (burned_mean, burned_lo, burned_hi))
    print(f"Year {y} baseline affected mean+CI:", (affected_mean, affected_lo, affected_hi))

    # Strategy: thinning
    thin_factor = 0.6
    forest_thin = forest.apply_thinning(thin_factor)
    mc_thin = MonteCarlo(forest=forest_thin, params=params, n_runs=n_runs, rng=np.random.default_rng(y + 1))

    report_thin = mc_thin.run()
    burned_arr_t, affected_arr_t = report_thin.convert_to_arrays()
    risk_thin = mc_thin.risk_map(n_runs=n_runs)
    results[(y, "thinning")] = (burned_arr_t, affected_arr_t, risk_thin)

    # Store p-values and effect sizes for later correction
    # (define these lists once before the loop)
    # p_burned = []; p_affected = []; delta_burned = []; delta_affected = []

    p_b, d_b = permutation_p_value(burned_arr, burned_arr_t, n_perm=5000, alternative="greater",
                                rng=np.random.default_rng(1000 + y))
    p_a, d_a = permutation_p_value(affected_arr, affected_arr_t, n_perm=5000, alternative="greater",
                                rng=np.random.default_rng(2000 + y))

    p_burned.append(p_b); delta_burned.append(d_b)
    p_affected.append(p_a); delta_affected.append(d_a)

    print(f"Year {y}: Delta burned={d_b:.4f}, p_raw_burned={p_b:.4g} | Delta affected={d_a:.4f}, p_raw_affected={p_a:.4g}")

    plt.figure(figsize=(6, 5))
    plt.hist(burned_arr_t, bins=25)
    plt.title(f"Year {y} thinning(factor={thin_factor}): burned fraction")
    plt.xlabel("burned fraction")
    plt.ylabel("count")
    plt.show()

    burned_mean_t, burned_lo_t, burned_hi_t = DataCollector.calculate_ci95_mean(burned_arr_t)
    affected_mean_t, affected_lo_t, affected_hi_t = DataCollector.calculate_ci95_mean(affected_arr_t)
    print(f"Year {y} thinning burned mean+CI:", (burned_mean_t, burned_lo_t, burned_hi_t))
    print(f"Year {y} thinning affected mean+CI:", (affected_mean_t, affected_lo_t, affected_hi_t))


# Summary line plot over time
baseline_means = []
thinning_means = []
for y in years_to_analyze:
    burned_b, _, _ = results[(y, "baseline")]
    burned_t, _, _ = results[(y, "thinning")]
    baseline_means.append(float(np.mean(burned_b)))
    thinning_means.append(float(np.mean(burned_t)))

plt.figure(figsize=(7, 4))
plt.plot(years_to_analyze, baseline_means, marker="o", label="baseline")
plt.plot(years_to_analyze, thinning_means, marker="o", label="thinning")
plt.title("Mean burned fraction over time (selected years)")
plt.xlabel("year")
plt.ylabel("mean burned fraction")
plt.legend()
plt.show()

## Baseline vs Thinning Analysis
We compare a baseline (no mitigation) scenario to a simple mitigation strategy:

- **Thinning (fuel reduction):** multiply the forest density everywhere by a constant factor `c < 1`
  (here `thin_factor = 0.6`).

Lower density reduces spread probability and can disconnect fuel pathways, reducing expected burned
and affected fractions.

In [ ]:
# Permutation test helpers (p-values) + Holm–Bonferroni correction
#
# We quantify thinning effectiveness with a nonparametric permutation test:
# - Statistic: difference in means, T = mean(baseline) - mean(thinning)
# - Alternative: thinning reduces outcomes, so we test H1: E[baseline] > E[thinning]
#
# We then correct across multiple years using Holm–Bonferroni to control family-wise error.

import numpy as np

def permutation_p_value(x, y, n_perm=5000, alternative="greater", rng=None):
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    if rng is None:
        rng = np.random.default_rng(0)

    t_obs = x.mean() - y.mean()
    pooled = np.concatenate([x, y])
    n_x = len(x)

    count = 0
    for _ in range(int(n_perm)):
        perm = rng.permutation(pooled)
        x_p = perm[:n_x]
        y_p = perm[n_x:]
        t = x_p.mean() - y_p.mean()

        if alternative == "greater":
            count += (t >= t_obs)
        elif alternative == "less":
            count += (t <= t_obs)
        else:
            count += (abs(t) >= abs(t_obs))

    return (count + 1) / (n_perm + 1), float(t_obs)


def holm_bonferroni(p_values, alpha=0.05):
    """Holm–Bonferroni multiple comparisons correction.

    Returns:
      - adjusted_p: Holm adjusted p-values (same order as input)
      - reject: whether each hypothesis is rejected at family-wise error rate alpha
    """
    p = np.asarray(p_values, dtype=float)
    m = len(p)
    order = np.argsort(p)
    p_sorted = p[order]

    # Holm adjusted p-values: adj_i = max_{j<=i} ((m-j)*p_(j)) with j 0-indexed
    adj_sorted = np.empty(m, dtype=float)
    running_max = 0.0
    for i, p_i in enumerate(p_sorted):
        adj_i = (m - i) * p_i
        running_max = max(running_max, adj_i)
        adj_sorted[i] = min(1.0, running_max)

    adjusted_p = np.empty(m, dtype=float)
    adjusted_p[order] = adj_sorted

    # Rejection decisions (step-down)
    reject_sorted = np.zeros(m, dtype=bool)
    for i, p_i in enumerate(p_sorted):
        if p_i <= alpha / (m - i):
            reject_sorted[i] = True
        else:
            break
    reject = np.empty(m, dtype=bool)
    reject[order] = reject_sorted

    return adjusted_p, reject

In [ ]:
# Risk map summaries over time (convert 2D risk maps into time-series lines)
#
# A risk map is a 2D field of per-cell probabilities. To plot "risk" as a line over time, we reduce
# each map to scalar summaries:
# - mean risk across the grid
# - upper-tail quantiles (p90, p99) to track hotspot intensity
# - fraction of cells above a threshold (e.g., risk > 0.5) to track high-risk area

years = list(years_to_analyze)

def risk_summaries(risk, threshold=0.5):
    r = np.asarray(risk, float).ravel()
    return {
        "mean": float(r.mean()),
        "p90": float(np.quantile(r, 0.90)),
        "p99": float(np.quantile(r, 0.99)),
        "frac_gt_t": float((r > threshold).mean()),
    }

baseline_risk_mean = []
thinning_risk_mean = []
baseline_risk_p90 = []
thinning_risk_p90 = []
baseline_risk_frac = []
thinning_risk_frac = []

thr = 0.5

for y in years:
    _, _, risk_b = results[(y, "baseline")]
    _, _, risk_t = results[(y, "thinning")]

    sb = risk_summaries(risk_b, threshold=thr)
    st = risk_summaries(risk_t, threshold=thr)

    baseline_risk_mean.append(sb["mean"])
    thinning_risk_mean.append(st["mean"])
    baseline_risk_p90.append(sb["p90"])
    thinning_risk_p90.append(st["p90"])
    baseline_risk_frac.append(sb["frac_gt_t"])
    thinning_risk_frac.append(st["frac_gt_t"])

plt.figure(figsize=(7,4))
plt.plot(years, baseline_risk_mean, marker="o", label="baseline")
plt.plot(years, thinning_risk_mean, marker="o", label="thinning")
plt.title("Mean risk over time")
plt.xlabel("Year")
plt.ylabel("Mean Risk")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

plt.figure(figsize=(7,4))
plt.plot(years, baseline_risk_p90, marker="o", label="baseline")
plt.plot(years, thinning_risk_p90, marker="o", label="thinning")
plt.title("90th percentile risk over time (hotspot intensity)")
plt.xlabel("Year")
plt.ylabel("Risk (90th percentile)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

plt.figure(figsize=(7,4))
plt.plot(years, baseline_risk_frac, marker="o", label="baseline")
plt.plot(years, thinning_risk_frac, marker="o", label="thinning")
plt.title(f"High-risk area over time (risk > {thr})")
plt.xlabel("Year")
plt.ylabel("Fraction of Cells")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Burned and Affected Area Analysis 

In [ ]:
# Burned + affected fractions over time (mean ± 95% CI)
#
# This is the main time-series result:
# - Burned fraction: fraction of cells that burn per run
# - Affected fraction: burned + adjacent-to-burned per run
#
# For each year and scenario (baseline vs thinning), we compute a mean and a normal-approximation 95% CI.

years = list(years_to_analyze)

def mean_ci(arr):
    m, lo, hi = DataCollector.calculate_ci95_mean(np.asarray(arr, float))
    return float(m), float(lo), float(hi)

burn_mean_b, burn_lo_b, burn_hi_b = [], [], []
burn_mean_t, burn_lo_t, burn_hi_t = [], [], []
aff_mean_b, aff_lo_b, aff_hi_b = [], [], []
aff_mean_t, aff_lo_t, aff_hi_t = [], [], []

for y in years:
    burned_b, affected_b, _ = results[(y, "baseline")]
    burned_t, affected_t, _ = results[(y, "thinning")]

    m, lo, hi = mean_ci(burned_b)
    burn_mean_b.append(m); burn_lo_b.append(lo); burn_hi_b.append(hi)

    m, lo, hi = mean_ci(burned_t)
    burn_mean_t.append(m); burn_lo_t.append(lo); burn_hi_t.append(hi)

    m, lo, hi = mean_ci(affected_b)
    aff_mean_b.append(m); aff_lo_b.append(lo); aff_hi_b.append(hi)

    m, lo, hi = mean_ci(affected_t)
    aff_mean_t.append(m); aff_lo_t.append(lo); aff_hi_t.append(hi)

plt.figure(figsize=(7,4))
plt.plot(years, burn_mean_b, marker="o", label="baseline")
plt.fill_between(years, burn_lo_b, burn_hi_b, alpha=0.2)
plt.plot(years, burn_mean_t, marker="o", label="thinning")
plt.fill_between(years, burn_lo_t, burn_hi_t, alpha=0.2)
plt.title("Burned fraction over time (mean ± 95% CI)")
plt.xlabel("Year")
plt.ylabel("Burned Fraction")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

plt.figure(figsize=(7,4))
plt.plot(years, aff_mean_b, marker="o", label="baseline")
plt.fill_between(years, aff_lo_b, aff_hi_b, alpha=0.2)
plt.plot(years, aff_mean_t, marker="o", label="thinning")
plt.fill_between(years, aff_lo_t, aff_hi_t, alpha=0.2)
plt.title("Affected fraction over time (mean ± 95% CI)")
plt.xlabel("Year")
plt.ylabel("Affected Fraction")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Significance Test of Thinning

In [ ]:
alpha = 0.05

p_b_adj, rej_b = holm_bonferroni(p_burned, alpha=alpha)
p_a_adj, rej_a = holm_bonferroni(p_affected, alpha=alpha)

print("\nMultiple comparisons correction (Holm–Bonferroni, per-metric across years)\n")
for i, y in enumerate(years_to_analyze):
    print(
        f"Year {y}: "
        f"Delta burned={delta_burned[i]:.4f}, p_adj_burned={p_b_adj[i]:.4g}, reject={bool(rej_b[i])} | "
        f"Delta affected={delta_affected[i]:.4f}, p_adj_affected={p_a_adj[i]:.4g}, reject={bool(rej_a[i])}"
    )

# Theoretical vs empirical comparison (density scaling sweeps)
This section connects the simulation to percolation/connectivity intuition.

Idea:
- Scale a fixed year's density map by a factor c in [0, 1] (uniform thinning).
- Estimate the expected burned fraction as a function of c.
- A nonlinear/transition-like curve supports the connectivity intuition.

Minimal robustness setup:
- Run the sweep for 3 representative years:
  * 2000 (baseline)
  * 2012 (gain breakpoint in our simplified time series)
  * 2019 (end of available lossyear range)
  
Note: This is not part of the thinning significance test; it's a qualitative/theory check.

In [ ]:
# Density scaling sweep (multi-year overlay)

base_years_for_sweep = (2000, 2012, 2019)

# Keep modest so runtime stays reasonable.
sweep_runs = 100
scales = np.linspace(0.1, 1.0, 10)


def run_density_scaling_sweep(base_year: int, scales, sweep_runs: int):
    """Run a uniform thinning sweep on a fixed base-year landscape.

    Returns:
      means: mean burned fraction at each scale
      ci_lo/ci_hi: 95% CI bounds for the mean at each scale
    """
    base_density = density_for_year(base_year)
    base_forest = Forest(base_density)

    means, ci_lo, ci_hi = [], [], []

    for c in scales:
        forest_c = base_forest.apply_thinning(float(c))
        mc_c = MonteCarlo(
            forest=forest_c,
            params=params,
            n_runs=sweep_runs,
            rng=np.random.default_rng(base_year * 10_000 + int(1_000 * c)),
        )

        report_c = mc_c.run()
        burned_c, _ = report_c.convert_to_arrays()
        mean_c, lo_c, hi_c = DataCollector.calculate_ci95_mean(burned_c)

        means.append(float(mean_c))
        ci_lo.append(float(lo_c))
        ci_hi.append(float(hi_c))

    return means, ci_lo, ci_hi


sweep_results = {y: run_density_scaling_sweep(y, scales=scales, sweep_runs=sweep_runs) for y in base_years_for_sweep}

plt.figure(figsize=(7, 4))
for y in base_years_for_sweep:
    means, lo, hi = sweep_results[y]
    plt.plot(scales, means, marker="o", label=f"year {y}")
    plt.fill_between(scales, lo, hi, alpha=0.15)

plt.title("Density scaling sweep (uniform thinning) across representative years")
plt.xlabel("Global density scale c (thinning factor)")
plt.ylabel("Burned fraction")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()